In [ ]:
########################################################################
# 1 : LOAD THE LIBRARIES AND SOURCE FILES
########################################################################

#library(tidyverse)
library(ggplot2)
library(plyr)
library(readr)
library(data.table)
library(dplyr)
library(Matrix)



# Custom functions
source("inputs/io_function.R") # CROOTBOX
source("inputs/getSUF.R") # MARSHAL


In [ ]:
# MARSHAL PARAMETERS

# E. We load the default parameter sets for the simulation 
psiCollar <- -15000
soil <- read_csv("inputs/soil.csv")
conductivities <- read_csv("inputs/conductivities.csv")

In [ ]:
# Root system

rootsystem <- fread("outputs/first_example.txt", header = T)

rootsystem %>%
  ggplot() +
  theme_classic() +
  geom_segment(aes(x = x1, y = z1, xend = x2, yend = z2), alpha=0.9) +
  coord_fixed()

In [ ]:
########################################################################
# 4 : RUN MARSHAL
########################################################################
# We loop over the input parameters vectors (see 2.D)

# Load the root system data you would like to use with marshal. 
# It is also possible to loop over a list of root systems is needed
      
# Run MARSHAL
hydraulics <- getSUF(rootsystem, conductivities, soil, psiCollar)

In [ ]:

hydraulic_archi <- hydraulics$root_system
hydraulic_archi$suf <- hydraulics$suf[,1]
hydraulic_archi$kr <- hydraulics$kr[,1]
hydraulic_archi$kx <- hydraulics$kx[,1]
hydraulic_archi$jr <- hydraulics$jr[,1]
hydraulic_archi$jxl <- hydraulics$jxl[,1]
hydraulic_archi$psi <- hydraulics$psi[,1]


print(paste0("KRS = ",hydraulics$krs))
print(paste0("Potential transpiration = ",hydraulics$tpot))
print(paste0("Actual transpiration = ",hydraulics$tact))


In [ ]:
hydraulic_archi %>%
  ggplot() +
  theme_classic() +
  geom_segment(aes(x = x1, y = z1, xend = x2, yend = z2, col = psi), alpha=0.9) +
  scale_color_viridis_c() + 
  coord_fixed()


In [ ]:

temp <- hydraulic_archi
temp$z1 <- round(temp$z1, 0)
temp$type <- factor(temp$type)

dens <- ddply(temp, .(z1, type), summarise, root = sum(suf))
yl <- "standart uptake fraction"

dens2 <- ddply(dens, .(z1), summarise, root = sum(root))
max <- dens2$z1[dens2$root == max(dens2$root)]

ggplot(data = dens, aes(x = z1, y = root, colour=type)) +
    theme_classic() +
    geom_line(data = dens, aes(x = z1, y = root, colour=type), alpha=0.5) +
    geom_vline(xintercept = max, lty=2) +
    stat_smooth(se=FALSE) +
    coord_flip() +
    xlab("depth (cm)") +
    ylab(yl)